# Title

In [177]:
import numpy as np
import pandas as pd
import os
import sqlalchemy

## Dataset

In [178]:
CWD = os.getcwd()
dataset_engine = sqlalchemy.create_engine(f"sqlite:///{CWD}/modeling_dataset.db")
dataset_engine.connect()

In [179]:
tables = pd.read_sql("SELECT * FROM sqlite_master WHERE type = 'table'", dataset_engine)
tables

,type,name,tbl_name,rootpage,sql
0,table,by,by,2,"CREATE TABLE ""by"" (\n\t""index"" BIGINT, \n\tlic..."
1,table,by-sa,by-sa,28,"CREATE TABLE ""by-sa"" (\n\t""index"" BIGINT, \n\t..."
2,table,by-nc,by-nc,410,"CREATE TABLE ""by-nc"" (\n\t""index"" BIGINT, \n\t..."
3,table,by-nc-sa,by-nc-sa,436,"CREATE TABLE ""by-nc-sa"" (\n\t""index"" BIGINT, \..."
4,table,by-nd,by-nd,1223,"CREATE TABLE ""by-nd"" (\n\t""index"" BIGINT, \n\t..."
5,table,by-nc-nd,by-nc-nd,1590,"CREATE TABLE ""by-nc-nd"" (\n\t""index"" BIGINT, \..."
6,table,publicdomain,publicdomain,803,"CREATE TABLE publicdomain (\n\t""index"" BIGINT,..."


In [180]:
webpages_dataset = pd.concat(
    [
        pd.read_sql(f"SELECT * FROM '{table_name}'", dataset_engine)
        for table_name in tables["name"]
    ]
)
webpages_dataset = webpages_dataset\
    .loc[webpages_dataset["contents"] != "", :]\
    .reset_index()\
    .drop(["index", "title", "level_0"], axis=1)
webpages_dataset.sample(5)

,license,url,contents
632,licenses/by-sa/2.0,https://www.azmirror.com/2022/04/20/states-rus...,States rush to revise their abortion laws as a...
1560,licenses/by-nc-sa/2.0,https://blogs.loc.gov/law/2015/07/the-basic-la...,The Basic Law of Hong Kong | In Custodia Legis...
1049,licenses/by-nc/1.0,https://ncp.nist.gov/checklist/725,NCP - Checklist CIS MongoDB Benchmark You are ...
1123,licenses/by-nc/2.0,https://www.archdaily.com/893167/these-are-the...,Gallery of These Are The Latin American Cities...
1033,licenses/by-nc/1.0,https://www.exploit-db.com/exploits/29516,Sucuri WebSite Firewall - Access Denied Websit...


In [181]:
webpages_dataset_deduplicate = webpages_dataset.groupby("url").first()\
    .reset_index()
webpages_dataset_deduplicate.describe()

,url,license,contents
count,1391,1391,1391
unique,1391,39,1347
top,http://archivescentral.org.nz/rights/CC-BY-NC-...,licenses/by/1.0,403 Forbidden 403 Forbidden nginx
freq,1,87,23


In [182]:
import dataset_sampling
license_map = dataset_sampling.get_license_map()
license_ser = pd.concat([v for v in license_map.values()])
license_ser_splits_df = license_ser.str.split("/", expand=True)
license_ser_splits_df = license_ser_splits_df.rename(
    columns = {
        0: "Tool Typing",
        1: "General Typing",
        2: "Version",
        3: "Jurisdiction"
    }
)
license_ser_splits_df["General Typing"] = license_ser_splits_df["General Typing"].str.replace("mark|zero", "publicdomain", regex=True)
license_ser_splits_df["General Typing"] = license_ser_splits_df["General Typing"].str.replace("by-nd-nc", "by-nc-nd", regex=True)
license_ser_splits_df["Version"] = license_ser_splits_df["Version"].astype(float)
license_one_hot_encoding = pd.DataFrame()
license_one_hot_encoding["by"] = license_ser_splits_df["General Typing"].str.contains("by")
license_one_hot_encoding["sa"] = license_ser_splits_df["General Typing"].str.contains("sa")
license_one_hot_encoding["nc"] = license_ser_splits_df["General Typing"].str.contains("nc")
license_one_hot_encoding["nd"] = license_ser_splits_df["General Typing"].str.contains("nd")
license_not_six_type = license_ser_splits_df["General Typing"].str.contains("by|sa|nc|nd")
license_one_hot_encoding["neither"] = ~(license_not_six_type.fillna(False))
license_df = pd.concat([license_ser, license_ser_splits_df, license_one_hot_encoding], axis = 1)\
    .rename(columns = {0: "license"})
license_df.head(6)

,license,Tool Typing,General Typing,Version,by,sa,nc,nd,neither
0,licenses/by/1.0,licenses,by,1.0,True,False,False,False,False
14,licenses/by/2.0,licenses,by,2.0,True,False,False,False,False
27,licenses/by/2.1,licenses,by,2.1,True,False,False,False,False
33,licenses/by/2.5,licenses,by,2.5,True,False,False,False,False
39,licenses/by/3.0,licenses,by,3.0,True,False,False,False,False
45,licenses/by/4.0,licenses,by,4.0,True,False,False,False,False


In [183]:
webpages_dataset_deduplicate = webpages_dataset_deduplicate.merge(license_df, on = "license")
webpages_dataset_deduplicate.sample(5)

,url,license,contents,Tool Typing,General Typing,Version,by,sa,nc,nd,neither
485,https://tsammalex.clld.org/parameters/psidiumg...,publicdomain/mark/1.0,Tsammalex -\n Psidium guajava (guav...,publicdomain,publicdomain,1.0,False,False,False,False,True
977,https://www.thomsontreks.com/blog/kilimanjaro-...,licenses/by-sa/2.5,403 Forbidden 403 Forbidden nginx,licenses,by-sa,2.5,True,True,False,False,False
725,https://collections.lib.utah.edu/ark:/87278/s6...,licenses/by-nc-sa/2.5,PediNeuroLogic Exam: Newborn: Normal: Tone - A...,licenses,by-nc-sa,2.5,True,True,True,False,False
815,https://podcasts.apple.com/us/podcast/cast-of-...,licenses/by/1.0,‎Cast of Many Things on Apple Podcasts Global ...,licenses,by,1.0,True,False,False,False,False
845,https://www.khronos.org/opencl/assets/CXX_for_...,licenses/by/1.0,The C++ for OpenCL 1.0 and 2021 Programming La...,licenses,by,1.0,True,False,False,False,False
2794,https://www.esa.int/Services/Creative_Commons_...,licenses/by-sa/2.0,ESA - Creative Commons Attribution-ShareAlike ...,licenses,by-sa,2.0,True,True,False,False,False


In [184]:
webpages_dataset_deduplicate.groupby("General Typing").count()

,url,license,contents,Tool Typing,Version,by,sa,nc,nd,neither
General Typing,,,,,,,,,,
by,402,402,402,402,402,402,402,402,402,402
by-nc,216,216,216,216,216,216,216,216,216,216
by-nc-nd,99,99,99,99,99,99,99,99,99,99
by-nc-sa,175,175,175,175,175,175,175,175,175,175
by-nd,113,113,113,113,113,113,113,113,113,113
by-sa,237,237,237,237,237,237,237,237,237,237
publicdomain,149,149,149,149,149,149,149,149,149,149


In [185]:
import string
def remove_unicodes(ser):
    return ser.map(lambda x: "".join([c for c in x if c in string.printable]))

In [186]:
webpages_dataset_deduplicate["parsed_contents"] = remove_unicodes(webpages_dataset_deduplicate["contents"])
webpages_dataset_deduplicate.loc[1149, ["contents", "parsed_contents"]]

contents           FALQs: Execution of Drug Offenders in Indonesi...
parsed_contents    FALQs: Execution of Drug Offenders in Indonesi...
Name: 1149, dtype: object

In [190]:
webpages_dataset_deduplicate["parsed_contents"] = \
    webpages_dataset_deduplicate["parsed_contents"].str.replace(
        r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(^rt)|(http.+?)|(www.+?)|(\d{3}\s*\w+\b)|([\n\t])|(\d)",
        " ",
        regex = True
    )

In [213]:
webpages_dataset_deduplicate[["contents", "parsed_contents"]].sample(10)

,contents,parsed_contents
453,DADUN: Associations between olfactory pathway ...,DADUN Associations between olfactory pathway ...
1108,403 Forbidden 403 Forbidden Access to this res...,Access to this resource on the server is d...
886,The Bee’s Knees. Curious Turns of Phrase #3 | ...,The Bees Knees Curious Turns of Phrase b...
48,Creative Commons Attribution 3.0 Unported (CC ...,Creative Commons Attribution Unported CC ...
646,Branch Retinal Vein Occlusion - EyeWiki Create...,Branch Retinal Vein Occlusion EyeWiki Create...
504,Some marijuana is being laced with meth and fe...,Some marijuana is being laced with meth and fe...
438,An Osmosis Video: Congestive Heart Failure (CH...,An Osmosis Video Congestive Heart Failure CH...
536,"""Household ecology and out-migration among eth...",Household ecology and out migration among eth...
991,🎈 Public Lab: bioassay Lead image: Sprouting l...,Public Lab bioassay Lead image Sprouting le...
1147,403 Forbidden 403 Forbidden nginx,nginx
